In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# 1. implement PCA on a dataset with 1000 rows and 4 features and map it to a space with 2 features with minimum loss of information. 

Get the covariance of the data manually for the given data. </b> 

In [2]:
Data = np.random.random_sample((1000,4))

### CODE HERE ###
X = Data
avg = np.mean(X,axis=0)
X = X-avg
        
covariance=np.dot(X.T,X)/(X.shape[0]-1)
print(covariance)

[[ 0.08513039 -0.0015167  -0.00280104  0.00116773]
 [-0.0015167   0.08288676 -0.00377977 -0.00335706]
 [-0.00280104 -0.00377977  0.08161504 -0.00546797]
 [ 0.00116773 -0.00335706 -0.00546797  0.08189721]]


Use the code in the following cell to get the eigen vectors and values. What should we do next to finish implementing PCA? Please leave comments for each part of your code. </b> 

In [3]:
value, vector = np.linalg.eig(covariance)
######################
### CODE HERE ###
n = 2
vector=vector.T
indexs=np.argsort(value)[::-1]
vector=vector[indexs]
value=value[indexs]
total = sum(value)
variance_of_each_feature = [(i / total)*100 for i in value]
features=vector[:n]

In [4]:
features

array([[-0.58698431,  0.09812121,  0.57202857, -0.56445102],
       [ 0.1890132 , -0.85239625,  0.46957102,  0.13114   ]])

# Q 2

## a) Mean Imputation 

In [5]:
# Create dataset
X, y = make_regression(n_samples=1000, n_features=5, random_state=0)
# Add missing values
X.ravel()[np.random.choice(X.size, 100, replace=False)] = np.nan
# Train test split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

- There are missing values in your dataset. Replace the missing values with the mean of the feature in your **training set**.
- Avinash use core Python, Numpy and Pandas functions

**HINT**: To ignore the NaNs in the mean computation use `np.nanmean` instead of `np.mean`

In [6]:
# Number of missing values in the training and the test set
sum(np.isnan(train_x.flatten())), sum(np.isnan(test_x.flatten()))

(80, 20)

In [7]:
### CODE HERE ###

col_avg = np.nanmean(train_x, axis=0)
inds = np.where(np.isnan(train_x))
train_x[inds] = np.take(col_avg, inds[1])

col_avg = np.nanmean(test_x, axis=0)
inds = np.where(np.isnan(test_x))
test_x[inds] = np.take(col_avg, inds[1])

<b> Train the model </b> 
- Just run the following cells, no need to do anything

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier

In [9]:
model = LinearRegression()
model.fit(train_x, train_y)
y_pred = model.predict(test_x)

## b) Compute Error Metrics 

$$
\mathrm{MSE}=\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}
$$

$$
\mathrm{MAE}=\frac{1}{n} \sum_{i=1}^{n} \mid y_{i}-\hat{y}_{i} \mid
$$

In [10]:
### CODE HERE ###
MSE = np.square(np.subtract(test_y,y_pred)).mean()
MAE = abs(np.subtract(test_y,y_pred)).mean()

print('MSE',MSE)
print('MAE',MAE)

MSE 184.36614936477704
MAE 4.066364168567996


# Q3. 

### Load `Iris` dataset using the below code**

In [11]:
from sklearn.datasets import load_iris
X, Y = load_iris(return_X_y=True)

**For this question, you need to apply 3-fold cross-validation on the IRIS dataset.**
    
**You CANNOT use any package that provides you a fast CV; 
instead, you have to implement it by yourself from scratch. 
You need to take all the steps related to CV implementation. 
What we expect to see at the end includes:**

* implementing the 3-fold CV correctly

* applying a decision tree classifier on each fold

* printing the ``precision`` of the model on each of the three folds. 

* reporting the ``mean`` and ``variance`` of the ``precision`` for these folds.

**Hints**: Using the below code, you can implement Decision Tree on any given dataset:
```python
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)
```
and if you use ``predict`` method, then you can see the predicted values given the trained model. 
```python
clf.predict(X_test)
```
<a href="https://scikit-learn.org/stable/modules/tree.html">[Source for help on DT]</a> No need to manually create your DT. Simply use the above package for Decision Tree.

For `precision` caluclation, you can use scikit-learn as well:

```python
from sklearn.metrics import precision_score
precision_score(y_true, y_pred, average='weighted')
```

In [12]:
### CODE HERE ###
from sklearn.metrics import precision_score
from sklearn import tree

class0_indx = np.where(Y==0)
class1_indx = np.where(Y==1)
class2_indx = np.where(Y==2)

n = int(np.ceil(len(class0_indx[0])/3))

folds_indx = []
start = 0
end = len(class0_indx[0])
i = n
while start<end:
    arr1 = class0_indx[0][start:i]
    arr2 = class1_indx[0][start:i]
    arr3 = class2_indx[0][start:i]
    indx = np.concatenate([arr1,arr2,arr3])
    folds_indx.append(indx)
    
    start = i
    i = i + n
    
#folds                                                              
fold1 = folds_indx[0]                                          
fold2 = folds_indx[1]
fold3 = folds_indx[2]

train1 = np.concatenate([fold1,fold2])
test1 = fold3

train2 = np.concatenate([fold1,fold3])
test2 = fold2

train3 = np.concatenate([fold2,fold3])
test3 = fold1

CV_Folds = [(train1,test1),(train2,test2),(train3,test3)]

stored_precision = []
for i,(train_index, test_index) in enumerate(CV_Folds):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    precision = precision_score(y_test, y_pred, average='weighted',zero_division=1)
    stored_precision.append(precision)
    print(f'Precision of model on fold-{i+1}',precision)
    
print('----------------------------------------')
print('Mean:',np.array(stored_precision).mean())
print('variance:',np.array(stored_precision).var())

Precision of model on fold-1 1.0
Precision of model on fold-2 0.9215686274509803
Precision of model on fold-3 0.9814814814814815
----------------------------------------
Mean: 0.9676833696441539
variance: 0.0011204406451248855
